<a href="https://colab.research.google.com/github/reallygooday/60daysofudacity/blob/master/Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



Colab Tricks:
https://rohitmidha23.github.io/Colab-Tricks/

https://colab.research.google.com/drive/1jPIq4hWxhG5KXulNst8ldAJMBtYeeUHb#scrollTo=gJini_448U8z

In [0]:
import numpy as np
import pandas as pd
import nltk
import spacy
from tqdm import tqdm
import os
import re
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [5]:
! pip install PyDrive

     |████████████████████████████████| 993kB 6.5MB/s 
  Created wheel for PyDrive: filename=PyDrive-1.3.1-cp36-none-any.whl size=27435 sha256=8d8b7a8544b26dd5b08b1fb0678f72c38e9adc6dde4368973978949eee82f198
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [7]:
from google.colab import files
uploaded = files.upload()

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [9]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [11]:
!ls '/content/drive/My Drive/text_classification_project'

negative  positve  text_classification_project.ipynb


In [0]:
# load file names
pos_files = os.listdir('/content/drive/My Drive/text_classification_project/positve/')
neg_files = os.listdir('/content/drive/My Drive/text_classification_project/negative/')

In [13]:
print(pos_files[:10])

['4737_7.txt', '11567_8.txt', '9279_7.txt', '3227_7.txt', '703_10.txt', '8279_10.txt', '74_8.txt', '11908_8.txt', '1636_10.txt', '12177_7.txt']


In [14]:
print(neg_files[:10])

[]


In [15]:
len(pos_files), len(neg_files)

(8743, 0)

In [0]:
# function to read text files
def read_review(filename):
    # open the file as read only
    file = open(filename, mode='rt', encoding='utf-8')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [0]:
# read and store positive reviews in a list
pos = []

for i in pos_files:
    pos.append(read_review("/content/drive/My Drive/text_classification_project/positve/"+i))
    
# read and store negative reviews in a list
neg = []

for i in neg_files:
    neg.append(read_review("/content/drive/My Drive/text_classification_project/negative/"+i))

In [0]:
df = pd.DataFrame({'review':(pos+neg)})

# '1' for positive reviews and '0' for negative reviews
df['class'] = 0
df['class'][:10000] = 1

In [0]:
# check class distribution
df['class'].value_counts()

In [0]:
df['review'][4]

In [0]:
replace_1 = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])")
replace_2 = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

# function to clean text data
def clean_reviews(reviews):
    reviews = [replace_1.sub("", line.lower()) for line in reviews]
    reviews = [replace_2.sub(" ", line) for line in reviews]
    return reviews

In [0]:
df['review'] = clean_reviews(df['review'])

In [0]:
df['review'][4]

In [0]:
from nltk import FreqDist

In [0]:
# function to plot top n most frequent words
def freq_words(x, terms = 30):
  all_words = ' '.join([text for text in x])
  all_words = all_words.split()
  
  fdist = FreqDist(all_words)
  words_df = pd.DataFrame({'word':list(fdist.keys()), 'count':list(fdist.values())})
  
  # selecting top n most frequent words
  d = words_df.nlargest(columns="count", n = terms) 
  plt.figure(figsize=(20,5))
  ax = sns.barplot(data=d, x= "word", y = "count")
  ax.set(ylabel = 'Count')
  plt.show()

In [0]:
freq_words(df['review'])

In [0]:
nlp = spacy.load('en_core_web_sm')

In [0]:
nlp.vocab['however'].is_stop = False
nlp.vocab['no'].is_stop = False
nlp.vocab['not'].is_stop = False
nlp.vocab['but'].is_stop = False

In [0]:
# tokenization using spaCy
def tokenization(x):
    reviews_tokens = []
    for i in tqdm(x):
        i = nlp(i)
        temp = []
        for j in i:
            temp.append(j.text)
        reviews_tokens.append(temp)
    
    return reviews_tokens

In [0]:
df['tokenized_review'] = tokenization(df['review'])

In [0]:
# function to remove stopwords
def strip_stopwords(reviews):
    s = []
    for r in tqdm(reviews):
        s_2 = []
        for token in r:
            if nlp.vocab[token].is_stop == True:
                continue
            else:
                s_2.append(token)
        s.append(" ".join(s_2))    
        
    return s

In [0]:
df['review_cleaned'] = strip_stopwords(df['tokenized_review'])

In [0]:
freq_words(df['review_cleaned'])

In [0]:
# remove 1 character terms (length == 1)
df['review_cleaned'] = df['review_cleaned'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>1]))

In [0]:
freq_words(df['review_cleaned'])